## __Lab 2: Naive Bayes Classifier__

__Lebohang Mosia : 2320396__ 

__Lusanda Mdhlalose : 2332720__ 

__Thapelo Duma : 2493083__

In [118]:
# !pip install nltk

In [119]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import numpy as np
import os
import pandas as pd
from collections import defaultdict, Counter

# nltk.download('punkt')
# nltk.download('punkt_tab')


In [120]:
input_file_path = "harry_potter(1)\\HP1.txt"

def clean_and_split_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    text = re.sub(r'[^\w\s]', '', text.lower())
    
    words = text.split()
    
    return words

words = clean_and_split_text(input_file_path)
# testWords = clean_and_split_text(input_test)

print(words)

['mr', 'and', 'mrs', 'dursley', 'of', 'number', 'four', 'privet', 'drive', 'were', 'proud', 'to', 'say', 'that', 'they', 'were', 'perfectly', 'normal', 'thank', 'you', 'very', 'much', 'they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense', 'mr', 'dursley', 'was', 'the', 'director', 'of', 'a', 'firm', 'called', 'grunnings', 'which', 'made', 'drills', 'he', 'was', 'a', 'big', 'beefy', 'man', 'with', 'hardly', 'any', 'neck', 'although', 'he', 'did', 'have', 'a', 'very', 'large', 'mustache', 'mrs', 'dursley', 'was', 'thin', 'and', 'blonde', 'and', 'had', 'nearly', 'twice', 'the', 'usual', 'amount', 'of', 'neck', 'which', 'came', 'in', 'very', 'useful', 'as', 'she', 'spent', 'so', 'much', 'of', 'her', 'time', 'craning', 'over', 'garden', 'fences', 'spying', 'on', 'the', 'neighbors', 'the', 'dursley', 's', 'had', 'a', 'small', 'son', 'called', 'd

# __Question 3__

In [121]:
def tokenize_words(words):
    return word_tokenize(' '.join(words))


# __Question 4__

In [122]:
def extract_ngrams(tokens, n):
    n_grams = list(ngrams(tokens, n))
    return n_grams

words = clean_and_split_text(input_file_path)
tokens = tokenize_words(words)
bigrams = extract_ngrams(tokens, 2)

print("Tokens:", tokens)
print("Bigrams:", bigrams)

Tokens: ['mr', 'and', 'mrs', 'dursley', 'of', 'number', 'four', 'privet', 'drive', 'were', 'proud', 'to', 'say', 'that', 'they', 'were', 'perfectly', 'normal', 'thank', 'you', 'very', 'much', 'they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense', 'mr', 'dursley', 'was', 'the', 'director', 'of', 'a', 'firm', 'called', 'grunnings', 'which', 'made', 'drills', 'he', 'was', 'a', 'big', 'beefy', 'man', 'with', 'hardly', 'any', 'neck', 'although', 'he', 'did', 'have', 'a', 'very', 'large', 'mustache', 'mrs', 'dursley', 'was', 'thin', 'and', 'blonde', 'and', 'had', 'nearly', 'twice', 'the', 'usual', 'amount', 'of', 'neck', 'which', 'came', 'in', 'very', 'useful', 'as', 'she', 'spent', 'so', 'much', 'of', 'her', 'time', 'craning', 'over', 'garden', 'fences', 'spying', 'on', 'the', 'neighbors', 'the', 'dursley', 's', 'had', 'a', 'small', 'son', 'cal

# Question 5

In [123]:
def read_pages(file_name, num_pages=None):
    words = clean_and_split_text(file_name)
    num_total_pages = len(words) // words_per_page + (1 if len(words) % words_per_page > 0 else 0)
    
    if num_pages is None or num_pages > num_total_pages:
        num_pages = num_total_pages
    
    pages = []
    for i in range(num_pages):
        start_index = i * words_per_page
        end_index = start_index + words_per_page
        page_words = words[start_index:end_index]
        tokens = tokenize_words(page_words)
        bigrams = extract_ngrams(tokens, 2)
        pages.append(bigrams)
    
    return pages

def split_data(pages):
    total_pages = len(pages)
    np.random.shuffle(pages)  # Shuffle pages to ensure random splits

    train_end = int(total_pages * training_page_percentage)
    validation_end = train_end + int(total_pages * val_page_percentage)
    
    train_pages = pages[:train_end]
    validation_pages = pages[train_end:validation_end]
    test_pages = pages[validation_end:]

    return train_pages, validation_pages, test_pages

def prepare_data(books_dir, book_names, num_pages_per_book=None):
    all_train_pages = []
    all_validation_pages = []
    all_test_pages = []

    for idx, book_name in enumerate(book_names, start=1):
        file_path = os.path.join(books_dir, book_name + ".txt")
        pages = read_pages(file_path, num_pages_per_book)

        train_pages, validation_pages, test_pages = split_data(pages)

        # Append data with corresponding targets
        all_train_pages.extend([(page, idx) for page in train_pages])
        all_validation_pages.extend([(page, idx) for page in validation_pages])
        all_test_pages.extend([(page, idx) for page in test_pages])

    return all_train_pages, all_validation_pages, all_test_pages

def create_dataframes(train_pages, validation_pages, test_pages, book_names):
    def create_dataframe(pages):
        rows = []
        for page, book_idx in pages:
            for ngram in page:
                rows.append({
                    'title': book_names[book_idx - 1],
                    'ngrams': ngram,
                    'page_number': pages.index((page, book_idx)) + 1,  # Adjust index as needed
                    'label': book_idx  # Label as the book index
                })
        return pd.DataFrame(rows)

    train_df = create_dataframe(train_pages)
    validation_df = create_dataframe(validation_pages)
    test_df = create_dataframe(test_pages)

    return train_df, validation_df, test_df


books_dir = "harry_potter(1)"
book_names = ["HP1", "HP2", "HP3", "HP4", "HP5", "HP6", "HP7"]
num_books = len(book_names)
training_page_percentage = 0.7
val_page_percentage = 0.1
testing_page_percentage = 0.2
words_per_page = 200
num_pages_per_book = 10

# Prepare data
train_pages, validation_pages, test_pages = prepare_data(books_dir, book_names, num_pages_per_book)

# Create dataframes
train_df, validation_df, test_df = create_dataframes(train_pages, validation_pages, test_pages, book_names)


# train_df.to_csv('train_data.csv', index=False)
# validation_df.to_csv('validation_data.csv', index=False)
# test_df.to_csv('test_data.csv', index=False)


In [124]:
print("Training DataFrame Sample:")
train_df.head(20)


Training DataFrame Sample:


,title,ngrams,page_number,label
0,HP1,"(mr, and)",1,1
1,HP1,"(and, mrs)",1,1
2,HP1,"(mrs, dursley)",1,1
3,HP1,"(dursley, of)",1,1
4,HP1,"(of, number)",1,1
5,HP1,"(number, four)",1,1
6,HP1,"(four, privet)",1,1
7,HP1,"(privet, drive)",1,1
8,HP1,"(drive, were)",1,1
9,HP1,"(were, proud)",1,1


In [125]:
train_df.tail(20)

,title,ngrams,page_number,label
9733,HP7,"(those, sitting)",49,7
9734,HP7,"(sitting, around)",49,7
9735,HP7,"(around, yaxley)",49,7
9736,HP7,"(yaxley, looked)",49,7
9737,HP7,"(looked, impressed)",49,7
9738,HP7,"(impressed, his)",49,7
9739,HP7,"(his, neighbor)",49,7
9740,HP7,"(neighbor, dolohov)",49,7
9741,HP7,"(dolohov, a)",49,7
9742,HP7,"(a, man)",49,7


In [126]:
print("Validation DataFrame Sample:")
validation_df.head(20)

Validation DataFrame Sample:


,title,ngrams,page_number,label
0,HP1,"(as, it)",1,1
1,HP1,"(it, was)",1,1
2,HP1,"(was, possible)",1,1
3,HP1,"(possible, to)",1,1
4,HP1,"(to, be)",1,1
5,HP1,"(be, the)",1,1
6,HP1,"(the, dursleys)",1,1
7,HP1,"(dursleys, shuddered)",1,1
8,HP1,"(shuddered, to)",1,1
9,HP1,"(to, think)",1,1


In [127]:
validation_df.tail(20)


,title,ngrams,page_number,label
1373,HP7,"(wroughtiron, gates)",7,7
1374,HP7,"(gates, barring)",7,7
1375,HP7,"(barring, the)",7,7
1376,HP7,"(the, mens)",7,7
1377,HP7,"(mens, way)",7,7
1378,HP7,"(way, neither)",7,7
1379,HP7,"(neither, of)",7,7
1380,HP7,"(of, them)",7,7
1381,HP7,"(them, broke)",7,7
1382,HP7,"(broke, step)",7,7


In [128]:
print("Test DataFrame Sample:")
test_df.head(20)

Test DataFrame Sample:


,title,ngrams,page_number,label
0,HP1,"(him, seized)",1,1
1,HP1,"(seized, his)",1,1
2,HP1,"(his, telephone)",1,1
3,HP1,"(telephone, and)",1,1
4,HP1,"(and, had)",1,1
5,HP1,"(had, almost)",1,1
6,HP1,"(almost, finished)",1,1
7,HP1,"(finished, dialing)",1,1
8,HP1,"(dialing, his)",1,1
9,HP1,"(his, home)",1,1


In [129]:
test_df.tail(20)

,title,ngrams,page_number,label
2766,HP7,"(gaze, had)",14,7
2767,HP7,"(had, wandered)",14,7
2768,HP7,"(wandered, upward)",14,7
2769,HP7,"(upward, to)",14,7
2770,HP7,"(to, the)",14,7
2771,HP7,"(the, body)",14,7
2772,HP7,"(body, revolving)",14,7
2773,HP7,"(revolving, slowly)",14,7
2774,HP7,"(slowly, overhead)",14,7
2775,HP7,"(overhead, and)",14,7


# __Question 6__

In [130]:
class NaiveBayesClassifier:
    def __init__(self):
        self.classes = None
        self.class_freq = None
        self.ngram_freq = None
        self.vocabulary = set()
        self.total_vocab_size = 0

    def train(self, train_data):
        self.classes = sorted(set(train_data['label']))
        self.class_freq = Counter(train_data['label'])
        self.ngram_freq = {cls: defaultdict(int) for cls in self.classes}
        
        for idx, row in train_data.iterrows():
            label = row['label']
            ngram = row['ngrams']
            self.vocabulary.add(ngram)
            self.ngram_freq[label][ngram] += 1
            
        self.total_vocab_size = len(self.vocabulary)

    def predict(self, page):
        log_probs = {cls: np.log(self.class_freq[cls] / sum(self.class_freq.values())) for cls in self.classes}

        for ngram in page:
            if ngram not in self.vocabulary:
                continue
            for cls in self.classes:
                term_freq = self.ngram_freq[cls][ngram]
                log_probs[cls] += np.log((term_freq + 1) / (self.class_freq[cls] + self.total_vocab_size))
        
        return max(log_probs, key=log_probs.get)

    def evaluate(self, test_data):
        correct_predictions = 0
        total_predictions = len(test_data)

        for idx, row in test_data.iterrows():
            page = row['ngrams']
            actual_label = row['label']
            predicted_label = self.predict([page])
            if predicted_label == actual_label:
                correct_predictions += 1

        accuracy = correct_predictions / total_predictions
        return accuracy

# Instantiate and train the classifier
nb_classifier = NaiveBayesClassifier()
nb_classifier.train(train_df)

# Evaluate on test data
accuracy = nb_classifier.evaluate(test_df)
accuracy


0.19526202440775306